In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_palette("GnBu_d")
import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime
from datetime import timedelta
import math
import random 
import timeit
from haversine import haversine
import sklearn as sk

In [2]:
brm = pd.read_csv("../resources/bremen.csv")
frb = pd.read_csv("../resources/freiburg.csv")
brm_hourly_driven = pd.read_csv("../resources/brm_hourly_driven.csv")


FileNotFoundError: [Errno 2] File ../resources/brm_hourly_driven.csv does not exist: '../resources/brm_hourly_driven.csv'

In [ ]:
brm_daily = brm.groupby("day").agg({"trip_duration":"sum","precipitation":"sum","temperature":"mean"})

In [ ]:
brm_daily["trip_duration"] = brm_daily["trip_duration"].total_seconds()

In [ ]:
x = brm_daily.values
x

In [ ]:
x = brm_daily.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
demand = pd.DataFrame(x_scaled)

In [ ]:
plt.plot(brm_daily["precipitation"])
plt.show()

In [ ]:
plt.plot(brm_daily["trip_duration"])
plt.show()

In [ ]:
plt.plot(brm_daily["temperature"])
plt.show()